# 전처리 코드(최종본을 올려놨으니 실행 안 해도 됩니다.)

### 행정동-직장인구 전처리

In [22]:
import pandas as pd

### 2024 4분기 직장인구 데이터 추출

# 1) 파일 경로
path = '서울시 상권분석서비스(직장인구-상권).csv'

# 2) 읽기 ― CP949(ANSI-K) 인코딩
df = pd.read_csv(path, encoding='cp949')

# 3) 2024년 4분기(20244)만 필터링
df_q4_2024 = df[df['기준_년분기_코드'] == 20244].copy()
#   ※ 혹시 문자열일 경우 df['기준_년분기_코드'].astype(str) == '20244'

# 4) 확인
print(df_q4_2024.head())
print(f'행 개수: {len(df_q4_2024):,d}')

# 5) 필요하면 저장
df_q4_2024.to_csv('직장인구_상권_2024Q4.csv',
                  index=False, encoding='utf-8-sig')


      기준_년분기_코드 상권_구분_코드 상권_구분_코드_명    상권_코드               상권_코드_명  총_직장_인구_수  \
3338      20244        R       전통시장  3130019               동대문상가D동          7   
3339      20244        A       골목상권  3110005                   세검정        544   
3340      20244        A       골목상권  3110023                 서울대병원         87   
3341      20244        D       발달상권  3120004                  종로구청       5275   
3342      20244        R       전통시장  3130018  동대문문구완구거리(동대문문구완구시장)        318   

      남성_직장_인구_수  여성_직장_인구_수  연령대_10_직장_인구_수  연령대_20_직장_인구_수  ...  \
3338           3           4               0               0  ...   
3339         306         238               0              91  ...   
3340          48          39               0              12  ...   
3341        2909        2366              72            1068  ...   
3342         142         176               0               0  ...   

      남성연령대_30_직장_인구_수  남성연령대_40_직장_인구_수  남성연령대_50_직장_인구_수  \
3338                 0              

In [23]:
import pandas as pd

### 직장인구 데이터와 행정동 데이터 join

# 1. 파일 경로와 인코딩
area_path = '서울시 상권분석서비스(영역-상권).csv'     # 상권 + 행정동·자치구 정보
job_path  = '직장인구_상권_2024Q4.csv'                # 2024년 4Q 직장인구

# ※ '영역-상권' CSV는 보통 CP949(ANSI-K),
#    직장인구 CSV는 UTF-8-SIG(앞서 저장)이라고 가정
area = pd.read_csv(area_path, encoding='cp949')
job  = pd.read_csv(job_path,  encoding='utf-8-sig')

# 2. 병합할 컬럼(행정동·자치구)만 추출
cols_admin = [
    '상권_코드',          # ← join key
    '자치구_코드', '자치구_코드_명',
    '행정동_코드', '행정동_코드_명'
]

# 상권코드 기준 left join
job_enriched = job.merge(
    area[cols_admin],
    on='상권_코드',
    how='left'
)

# 3. 결과 확인
print(job_enriched.head())
print(f'\n행 개수: {len(job_enriched):,d}  |  컬럼 개수: {job_enriched.shape[1]}')

# 4. 저장(UTF-8-SIG: Excel 한글 깨짐 방지)
out_path = '직장인구_상권_2024Q4_with_admin.csv'
job_enriched.to_csv(out_path, index=False, encoding='utf-8-sig')

   기준_년분기_코드 상권_구분_코드 상권_구분_코드_명    상권_코드               상권_코드_명  총_직장_인구_수  \
0      20244        R       전통시장  3130019               동대문상가D동          7   
1      20244        A       골목상권  3110005                   세검정        544   
2      20244        A       골목상권  3110023                 서울대병원         87   
3      20244        D       발달상권  3120004                  종로구청       5275   
4      20244        R       전통시장  3130018  동대문문구완구거리(동대문문구완구시장)        318   

   남성_직장_인구_수  여성_직장_인구_수  연령대_10_직장_인구_수  연령대_20_직장_인구_수  ...  \
0           3           4               0               0  ...   
1         306         238               0              91  ...   
2          48          39               0              12  ...   
3        2909        2366              72            1068  ...   
4         142         176               0               0  ...   

   여성연령대_10_직장_인구_수  여성연령대_20_직장_인구_수  여성연령대_30_직장_인구_수  여성연령대_40_직장_인구_수  \
0                 0                 0                 1  

In [24]:
import pandas as pd
import re

### 자치구, 행정동 별 직장인구 그룹화

# 1. 원본 CSV 읽기
src = '직장인구_상권_2024Q4_with_admin.csv'   # ← 경로 알맞게 수정
df  = pd.read_csv(src, encoding='utf-8-sig')

# 2. ‘남성’ 또는 ‘여성’이 들어간 모든 컬럼 제거
#    (총 남성·여성 + 성별×연령대 컬럼까지 한꺼번에 제외)
gender_pat   = re.compile(r'(남성|여성)')
gender_cols  = [c for c in df.columns if gender_pat.search(c)]
df = df.drop(columns=gender_cols)

# 3. 필요 없는 식별 컬럼 제거
#    ─ 행정동·자치구 “코드” / 상권 관련 식별자 / 기준년분기
cols_to_remove = [
    '행정동_코드',
    '자치구_코드',
    '기준_년분기_코드',
    '상권_구분_코드', '상권_구분_코드_명',
    '상권_코드',       # 상권 코드 자체도 결과에 포함하지 않음
    '상권_코드_명'
]
df = df.drop(columns=[c for c in cols_to_remove if c in df.columns])

# 4. 컬럼명 간단히 변경 (선택)
#    ─ 자치구_코드_명 → 자치구, 행정동_코드_명 → 행정동
df = df.rename(columns={
    '자치구_코드_명': '자치구',
    '행정동_코드_명': '행정동'
})

# 5. 자치구·행정동별 총 직장인구 + 연령대별 직장인구 합계
group_keys = ['자치구', '행정동']
value_cols = [c for c in df.columns if c not in group_keys]

dong_summary = (
    df.groupby(group_keys, as_index=False)[value_cols]
      .sum()
      .sort_values('총_직장_인구_수', ascending=False)
)

# 6. 결과 저장 (UTF-8-SIG → Excel / VS Code 한글 깨짐 방지)
out = '2024Q4_직장인구_행정동.csv'
dong_summary.to_csv(out, index=False, encoding='utf-8-sig')


### 행정동-추정매출 전처리

In [25]:
import pandas as pd
import re

# ─────────────────────────────────────────────
# 1. 파일 경로와 인코딩
# ─────────────────────────────────────────────
sales_path = '서울시 상권분석서비스(추정매출-상권)_2024년.csv'
area_path  = '서울시 상권분석서비스(영역-상권).csv'

# 추정매출 CSV는 대개 CP949, 최근 파일은 UTF-8인 경우도 있습니다.
# 인코딩이 다르면 encoding='utf-8'로 바꿔 보세요.
sales = pd.read_csv(sales_path, encoding='utf-8')
area  = pd.read_csv(area_path,  encoding='cp949')

# ─────────────────────────────────────────────
# 2. 2024년 4분기(기준_년분기_코드 = 20244) 필터링
# ─────────────────────────────────────────────
sales = sales[sales['기준_년분기_코드'] == 20244].copy()

# ─────────────────────────────────────────────
# 3. 상권 ↔ 행정동 · 자치구 정보 조인
# ─────────────────────────────────────────────
# area 파일에서 필요한 열만 추출
area_sub = area[['상권_코드', '자치구_코드_명', '행정동_코드_명']].rename(columns={
    '자치구_코드_명': '자치구',
    '행정동_코드_명': '행정동'
})

sales = sales.merge(area_sub, on='상권_코드', how='left')

# ─────────────────────────────────────────────
# 4. 집계 대상(매출) 컬럼 목록 만들기
#    - 남성·여성 구분 컬럼은 제외
# ─────────────────────────────────────────────
value_cols = [
    c for c in sales.columns
    if ('매출' in c and '금액' in c)          # 매출 금액 관련
       and not re.search(r'(남성|여성)', c)   # 성별 제외
]

# 예시로 반드시 포함되는 주요 컬럼
must_have = [
    '당월_매출_금액', '주중_매출_금액', '주말_매출_금액'
]
missing = [c for c in must_have if c not in value_cols]
if missing:
    raise ValueError(f"다음 필수 컬럼을 찾을 수 없습니다: {missing}")

# ─────────────────────────────────────────────
# 5. NaN → 0으로 바꾼 뒤 자치구·행정동별 합계
# ─────────────────────────────────────────────
sales[value_cols] = sales[value_cols].fillna(0)

summary = (
    sales.groupby(['자치구', '행정동'], as_index=False)[value_cols]
         .sum()
         .sort_values('당월_매출_금액', ascending=False)
)

# ─────────────────────────────────────────────
# 6. 결과 저장 (UTF-8-SIG: Excel 한글 깨짐 방지)
# ─────────────────────────────────────────────
out_path = '2024Q4_추정매출_자치구_행정동.csv'
summary.to_csv(out_path, index=False, encoding='utf-8-sig')


### 행정동-상권변화지표 전처리

In [26]:
import pandas as pd

## 2024 4분기 데이터 추출

# 1. 파일 경로
src_path = '서울시 상권분석서비스(상권변화지표-행정동).csv'

# 2. CSV 읽기  (CP949 → 실패 시 UTF-8-SIG 재시도)
try:
    df = pd.read_csv(src_path, encoding='cp949')
except UnicodeDecodeError:
    df = pd.read_csv(src_path, encoding='utf-8-sig')

# 3. 2024년 4분기만 추출 (기준_년분기_코드 == 20244)
df_q4_2024 = df[df['기준_년분기_코드'] == 20244].copy()

# 4. 결과 확인 (선택)
print(df_q4_2024.head())            # 앞 5행 미리보기
print(f'\n행 개수: {len(df_q4_2024):,d}')

# 5. 저장
out_path = '상권변화지표_행정동_2024Q4.csv'
df_q4_2024.to_csv(out_path, index=False, encoding='utf-8-sig')


     기준_년분기_코드    행정동_코드 행정동_코드_명 상권_변화_지표 상권_변화_지표_명  운영_영업_개월_평균  \
484      20244  11110515    청운효자동       LH       상권확장          100   
730      20244  11110530      사직동       HH         정체          119   
731      20244  11110540      삼청동       LH       상권확장          100   
732      20244  11110550      부암동       HH         정체          113   
736      20244  11110560      평창동       HH         정체          119   

     폐업_영업_개월_평균  서울_운영_영업_개월_평균  서울_폐업_영업_개월_평균  
484           55             109              52  
730           59             109              52  
731           57             109              52  
732           58             109              52  
736           61             109              52  

행 개수: 425


In [27]:
import pandas as pd

## 자치구 컬럼 추가

# 1. 파일 경로
dong_path   = '상권변화지표_행정동_2024Q4.csv'         # ④분기로 추린 결과
area_path   = '서울시 상권분석서비스(영역-상권).csv'   # 자치구 코드·명 포함

# 2. CSV 읽기
#    (서울열린데이터광장 CSV는 CP949 → 에러 나면 utf-8 시도)
dong = pd.read_csv(dong_path, encoding='utf-8-sig')        # 방금 저장한 파일은 UTF-8
try:
    area = pd.read_csv(area_path, encoding='cp949')
except UnicodeDecodeError:
    area = pd.read_csv(area_path, encoding='utf-8-sig')

# 3. 자치구 코드 ↔ 자치구명 매핑 테이블 만들기
gu_map = (
    area[['자치구_코드', '자치구_코드_명']]
        .drop_duplicates()
        .rename(columns={'자치구_코드_명': '자치구'})
)

# 4. 행정동 데이터에 자치구 코드 파생 → 자치구명 조인
# 4-1) 행정동 코드에서 앞 5자리(자치구 코드) 추출
dong['자치구_코드'] = dong['행정동_코드'].astype(str).str[:5].astype(int)

# 4-2) 매핑 조인
dong = dong.merge(gu_map, on='자치구_코드', how='left')

# 5. 컬럼 정리(선택) —— 자치구 코드가 필요 없으면 제거
dong = dong.drop(columns=['자치구_코드'])

# 6. 저장
out_path = '상권변화지표_자치구_행정동_2024Q4.csv'
dong.to_csv(out_path, index=False, encoding='utf-8-sig')


In [ ]:
import pandas as pd
from pathlib import Path

# 1. 파일 경로
src = Path('상권변화지표_자치구_행정동_2024Q4.csv')

# 2. CSV 읽기 (UTF-8 우선, 실패 시 CP949)
try:
    df = pd.read_csv(src, encoding='utf-8-sig')
except UnicodeDecodeError:
    df = pd.read_csv(src, encoding='cp949')

# 3. 컬럼명 정리
#    ─ '행정동_코드_명' → '행정동' 으로 변경
df = df.rename(columns={'행정동_코드_명': '행정동'})

# 4. 불필요 행 제거
df = df[df['행정동'] != '일원2동']

# ─────────────────────────────────────────────
# 5. 불필요 컬럼 제거
# ─────────────────────────────────────────────
drop_cols = ['기준_년분기_코드', '행정동_코드']
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

# ─────────────────────────────────────────────
# 6. 컬럼 재배열  (자치구 → 나머지 순서)
# ─────────────────────────────────────────────
priority = ['자치구', '행정동']
priority_existing = [c for c in priority if c in df.columns]
remaining = [c for c in df.columns if c not in priority_existing]

df = df[priority_existing + remaining]

# ─────────────────────────────────────────────
# 7. 저장 (원래 파일명 덮어쓰기)
# ─────────────────────────────────────────────
out_path = Path('2024Q4_상권변화지표_자치구_행정동.csv')
df.to_csv(out_path, index=False, encoding='utf-8-sig')


### 행정동 코드 추가

In [33]:
# -*- coding: utf-8 -*-
import pandas as pd
import re
from pathlib import Path

root = Path('./')

excel_path = root / '센서스 공간정보 지역 코드.xlsx'
csv_paths  = [
    root / '2024Q4_상권변화지표_자치구_행정동.csv',
    root / '2024Q4_직장인구_행정동.csv',
    root / '2024Q4_추정매출_자치구_행정동.csv'
]

# ────────────────────────────────
# 0. 행정동 문자열 정규화 함수
# ────────────────────────────────
def norm(name: str) -> str:
    if pd.isna(name):
        return name
    name = str(name).strip()
    name = re.sub(r'[·∙ㆍ]', '?', name)
    return name

# ────────────────────────────────
# 1. 서울 행정동 코드 매핑
# ────────────────────────────────
df_map = (
    pd.read_excel(excel_path, sheet_name=0, header=1, dtype=str)
      .rename(columns={'읍면동명칭': '행정동'})
)

for col, w in [('시도코드', 2), ('시군구코드', 3), ('읍면동코드', 3)]:
    df_map[col] = df_map[col].str.zfill(w)

df_map = df_map[df_map['시도코드'] == '11'].copy()          # 서울만
df_map['행정동_코드'] = df_map['시도코드'] + df_map['시군구코드'] + df_map['읍면동코드']
df_map['행정동_norm'] = df_map['행정동'].apply(norm)

mapping = df_map[['행정동_norm', '행정동_코드']].drop_duplicates()
print(f"서울 매핑 테이블: {len(mapping):,}개 행정동")

# ────────────────────────────────
# 2. CSV 처리
# ────────────────────────────────
for p in csv_paths:
    try:
        df = pd.read_csv(p, encoding='utf-8-sig')
    except UnicodeDecodeError:
        df = pd.read_csv(p, encoding='cp949')

    if '행정동' not in df.columns:
        raise KeyError(f"{p.name}: '행정동' 컬럼이 없습니다.")

    # 2-A. ‘상일동’ → 상일1동·상일2동 복제
    mask = df['행정동'].str.strip() == '상일동'
    if mask.any():
        rows = df[mask]
        row1 = rows.copy(); row1['행정동'] = '상일1동'
        row2 = rows.copy(); row2['행정동'] = '상일2동'
        df = pd.concat([df[~mask], row1, row2], ignore_index=True)
        print(f"{p.name} : 상일동 → 상일1·2동으로 복제 {len(rows)}행 → 2배")

    # 2-B. 기존 코드 컬럼 제거
    df = df.drop(columns=[c for c in df.columns if c.startswith('행정동_코드')],
                 errors='ignore')

    # 2-C. 정규화 컬럼
    df['행정동_norm'] = df['행정동'].apply(norm)

    # 2-D. 매핑 조인
    df_out = df.merge(mapping, on='행정동_norm', how='left')
    df_out = df_out.drop(columns=['행정동_norm'])

    miss = df_out['행정동_코드'].isna().sum()
    print(f"{p.name} ▶ 매칭 실패 {miss}행 / {len(df_out)}")

    # 2-E. 새 파일 저장
    new_path = p.with_stem(p.stem + '_최종')
    df_out.to_csv(new_path, index=False, encoding='utf-8-sig')
    print(f"  ↳ 저장: {new_path.name}")

print("🎉  모든 CSV: 가운뎃점 정규화 + 상일동 분할 + 새 코드 추가 완료")


서울 매핑 테이블: 426개 행정동
2024Q4_상권변화지표_자치구_행정동.csv : 상일동 → 상일1·2동으로 복제 1행 → 2배
2024Q4_상권변화지표_자치구_행정동.csv ▶ 매칭 실패 1행 / 428
  ↳ 저장: 2024Q4_상권변화지표_자치구_행정동_최종.csv
2024Q4_직장인구_행정동.csv : 상일동 → 상일1·2동으로 복제 1행 → 2배
2024Q4_직장인구_행정동.csv ▶ 매칭 실패 0행 / 403
  ↳ 저장: 2024Q4_직장인구_행정동_최종.csv
2024Q4_추정매출_자치구_행정동.csv : 상일동 → 상일1·2동으로 복제 1행 → 2배
2024Q4_추정매출_자치구_행정동.csv ▶ 매칭 실패 0행 / 401
  ↳ 저장: 2024Q4_추정매출_자치구_행정동_최종.csv
🎉  모든 CSV: 가운뎃점 정규화 + 상일동 분할 + 새 코드 추가 완료
